In [ ]:
import girder_client as gc
import json
import collections
import pymongo        
BASE_URL = 'https://isic-archive.com/girder/api/v1'
gc = gc.GirderClient(apiUrl=BASE_URL)

In [2]:
### I hard coded the LesionImageFolder ID--- this is where all the Annotaiton Data lives
AnnotationStudiesCollectionId = '55943cff9fc3c13155bcad61'

# LesionImageFolderId='573f11119fc3c132505c0ee7'  #<<<This is the FIRST STUDY ACTUALLY
# LIFdata = gc.listFolder(LesionImageFolderId)

AnnotationStudies = gc.listFolder(AnnotationStudiesCollectionId,parentFolderType='Collection')

In [49]:
### Get folderInfo for a rater.... this stores what images they marked up
## I want to have a nested dictionary as follows   STUDY_NAME --> IMAGE_NAME --> ANNOTATORS
ReformmedStudyDict = {}


fieldsToRemove =  ['size','creatorId','baseParentId','license','description','_modelType','folderId', 'baseParentType','created', '_id']


metaFieldsToRemove = ['studyId','userId','imageId']


for cs in AnnotationStudies:
    CurStudyName = cs['name']
    print CurStudyName,"is being Analyzed"
    if CurStudyName not in ReformmedStudyDict:
        ReformmedStudyDict[CurStudyName] = {}


    ## Now get the data for the current Study
    LIFdata = gc.listFolder(cs['_id'])
    
    
    for LIF in LIFdata:
        curUserName = LIF['name'].split()[0]
        print curUserName,"is being processed"
        curUserAnnotationList = gc.listItem( LIF['_id'])

        for cua in curUserAnnotationList:
            imgName =  cua['name']
            annotationInfo = cua['meta']
            ### I'm going to strip out keys I don't need?
            ### Need to see if this Image has been seen
            if imgName not in ReformmedStudyDict[CurStudyName]:
                ReformmedStudyDict[CurStudyName][imgName] = {}
                ReformmedStudyDict[CurStudyName][imgName]['raters'] = {}
           
            try:
                ## See if current annotation has a status of ok
                curStatus = cua['meta']['status']
                #print curStatus
                ## if this key is not there, means the user never completed the nanotations
                
            except:
                break
                
            if (curUserName not in ReformmedStudyDict[CurStudyName][imgName]['raters']):
                cuaClean = cua.copy()
                
                for ftr in fieldsToRemove:
                    if ftr in cuaClean:
                        del cuaClean[ftr]

                for ftr in metaFieldsToRemove:
                    if ftr in cuaClean['meta']:
                        del cuaClean['meta'][ftr]
                ### now clean up the metadata object as well.. too much data is being returend
                stop=start=None
                #meta fields to delete
                if 'stopTime' in cuaClean['meta']:
                    stop = dateutil.parser.parse(cuaClean['meta']['stopTime'])
                    del cuaClean['meta']['stopTime']
                if 'startTime' in cuaClean['meta']:
                    start = dateutil.parser.parse(cuaClean['meta']['startTime'])
                if stop and start:

                    cuaClean['meta']['markupTime'] = "%s" % (stop-start)
                
                
                ReformmedStudyDict[CurStudyName][imgName]['raters'][curUserName] = cuaClean

                
with open('momo.json','w') as fp:
    json.dump(ReformmedStudyDict,fp,indent=1)

ISBI Lesion Classification is being Analyzed
carrerac is being processed
halperna is being processed
jldefazio is being processed
marghooa is being processed
njaimes is being processed
oyelamos is being processed
quigleye is being processed
scopea1 is being processed
UDA-1 Testing is being Analyzed
aadikalloo is being processed
brianhelba is being processed
davidmanthey is being processed
konstantinosliopyris is being processed
msmolens is being processed


In [26]:
import datetime as dt

stopTime =  "2016-08-25T17:54:55.409000+00:00"

!pip install python-dateutil==2.4.2

import dateutil.parser
st = dateutil.parser.parse(stopTime)
print st

You are using pip version 8.1.2, however version 9.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
2016-08-25 17:54:55.409000+00:00


[u'UDA-1 Testing', u'ISBI Lesion Classification']


In [16]:
# print ReformmedStudyDict.keys()
# print ReformmedStudyDict['ISBI Lesion Classification'].keys()

for studyName,studyAnnotationData in ReformmedStudyDict.iteritems():
    print "------%s----" % studyName
    for img in studyAnnotationData:
        print img, len(studyAnnotationData[img]['raters'])

with open('momo.json','w') as fp:
    json.dump(ReformmedStudyDict,fp,indent=2)
        

------UDA-1 Testing----
ISIC_0000169 5
ISIC_0000297 5
ISIC_0000316 5
ISIC_0000160 5
ISIC_0000166 5
ISIC_0000315 5
ISIC_0000313 5
ISIC_0000262 5
ISIC_0000295 5
ISIC_0000199 5
ISIC_0000235 5
ISIC_0000319 5
ISIC_0000296 5
ISIC_0000210 5
ISIC_0000314 5
ISIC_0000212 5
ISIC_0000190 5
ISIC_0000311 5
ISIC_0000310 5
ISIC_0000238 5
ISIC_0000312 5
ISIC_0000273 5
ISIC_0000252 5
ISIC_0000253 5
ISIC_0000306 5
ISIC_0000304 5
ISIC_0000170 5
ISIC_0000173 5
ISIC_0000158 5
ISIC_0000298 5
ISIC_0000299 5
ISIC_0000178 5
ISIC_0000213 5
ISIC_0000198 5
ISIC_0000318 5
ISIC_0000320 5
ISIC_0000209 5
ISIC_0000269 5
ISIC_0000187 5
ISIC_0000302 5
ISIC_0000303 5
ISIC_0000300 5
ISIC_0000301 5
ISIC_0000243 5
ISIC_0000307 5
ISIC_0000241 5
ISIC_0000305 5
ISIC_0000308 5
ISIC_0000309 5
ISIC_0000317 5
------ISBI Lesion Classification----
ISIC_0000022 8
ISIC_0000287 8
ISIC_0000040 8
ISIC_0000510 8
ISIC_0000311 8
ISIC_0000289 8
ISIC_0000141 8
ISIC_0000164 8
ISIC_0000518 8
ISIC_0000499 8
ISIC_0009955 8
ISIC_0009958 8
ISIC_0009

In [ ]:
print len(LIFdata) ## These are the 8 raters.. what I care about is the data from each rater..

LIFdataClean = []
for lf in  LIFdata:
    #print lf.keys()
#    [u'size', u'updated', u'description', u'created', u'baseParentType', u'public', u'meta', u'baseParentId', u'creatorId', u'parentId', u'parentCollection', u'_id', u'_modelType', u'_accessLevel', u'name']
    keysToKeep = [  u'meta', u'baseParentId',  u'parentId', u'_id',u'name']
    
    lfClean = lf.copy()
    for k,v in lf.iteritems():
        if k not in keysToKeep:
            del lfClean[k]
    LIFdataClean.append(lfClean)
    
#     for k,v in lf.iteritems():
#         if k in keysToKeep:
#             print k,v
#     sys.exit()
print LIFdataClean

In [ ]:
from pprint import pprint

In [ ]:
### The data is currently nearly impossible to decipher based on how it's stored
### I want to basically know the Study Names, the images in that study, the people who rated it, and then their answers...

### First look in the Annotation Studies Folder
AnnotStudyColl = '55943cff9fc3c13155bcad61'
AnnotStudies = gc.listFolder(AnnotStudyColl,parentFolderType='collection')



In [ ]:
AnnotStudiesClean = []
for ast in AnnotStudies:
    AnnotStudiesClean.append( {'_id': ast['_id'], 'studyId': ast['_id'], 'studyName': ast['name'], 
                             'studyFeatureSetId': ast['meta']['featuresetId']} )

In [ ]:
pprint( AnnotStudiesClean)

featureSetList = gc.getResource('featureset')


In [ ]:
featureSetData = {}
for fsl in featureSetList:
    curFeatSet = gc.getResource('featureset/%s' % fsl['_id'])
#     print curFeatSet
#     print fsl['name']
    featureSetData[fsl['name']] = curFeatSet

In [ ]:
print featureSetData.keys()

In [ ]:
pmc = pymongo.MongoClient('localhost',27017)
pmc.database_names()
ISIC_DB = pmc['ISIC_DB']

In [ ]:
https://isic-archive.com/girder#collection/55943cff9fc3c13155bcad61/folder/573f11119fc3c132505c0ee7